On va tenter d'utiliser l'approche la plus basique pour classifier les crimes : un classifieur knn.

La bibliothèque scikit learn en propose une implémentation simple d'utilisation. Les paramètres sur lesquels nous jouerons sont le nombre de voisins dans le knn, et la distance utilisée pour rapprocher-éloigner les éléments du datasets.

Plus précisément nous utiliserons la norme $\ell_p$ en jouant sur $p$.

In [ ]:
# %cd /content/drive/MyDrive/Ponts/MachineLearning/CrimeSF_Malap

/content/drive/MyDrive/Ponts/MachineLearning/CrimeSF_Malap


In [ ]:
import pandas as pd
import numpy as np


On récupère les données qu'on a prétraitées, on travaille sur un sample pour la détermination des paramètres optimaux.

In [ ]:
train_data= pd.read_csv('data/pre_processing_train_data.csv')
train_data=train_data.iloc[:,1:]

In [ ]:
train_sample = train_data.sample(n=20000)
train_labels=train_sample['Category']
train_sample.drop('Category',inplace =True, axis=1)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss

In [ ]:
X = train_sample
y = train_labels

On crée manuellement un jeu d'entraînement et un jeu de validation pour le modèle.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=9, p=1)
knn.fit(X_train, y_train)

p_pred = knn.predict_proba(X_test)

print(accuracy_score(y_test, np.argmax(p_pred,axis=1)))

0.15125


Maintenant on performe une cross-val sur le nombre de voisins, et sur la distance utilisée (de la forme $||x-y||_p$)

In [ ]:
def cross_val_knn(X,y,nb_range,p_range):
    n=len(nb_range)
    m=len(p_range)
    res=np.zeros((n,m))
    for i in range(n):
        for j in range(m):
            print((nb_range[i],p_range[j]))
            knn = KNeighborsClassifier(n_neighbors=nb_range[i], p=p_range[j])
            scores=cross_val_score(knn,X,y)
            score=np.mean(scores)
            print(score)
            res[i,j]=score
    (imax,jmax)=np.unravel_index(res.argmax(), res.shape)
    return (nb_range[imax],p_range[jmax])

n_voisins, p_opti = cross_val_knn(X,y,nb_range=range(1,21),p_range=range(1,6))


On retient donc le couple $n_{voisins} = 20$ et p = 3.

Le fait de prendre autant de voisins va totalement sacrifier les classes peu représentées.

Maintenant on entraîne un knn sur l'ensemble des data

In [ ]:
train_data= pd.read_csv('data/pre_processing_train_data.csv')
train_data=train_data.iloc[:,1:]
train_labels=train_data['Category']
train_data.drop('Category',inplace =True, axis=1)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=20, p=3)
knn.fit(train_data, train_labels)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=20, p=3,
                     weights='uniform')

In [ ]:
test_data = pd.read_csv('data/pre_processing_test_data.csv')
test_data = test_data.iloc[:,1:]


In [ ]:
p_pred = knn.predict_proba(test_data)
prediction = np.argmax(p_pred,axis=1)


On sauvegarde nos prédictions dans un .npy  pour ne pas avoir à réentraîner le réseau.

In [ ]:
np.save('results/prediction.npy', p_pred)

In [ ]:
print(p_pred)

[[0.05 0.25 0.15 ... 0.   0.   0.  ]
 [0.05 0.25 0.15 ... 0.   0.   0.  ]
 [0.05 0.25 0.2  ... 0.   0.   0.  ]
 ...
 [0.05 0.05 0.05 ... 0.   0.   0.  ]
 [0.05 0.05 0.05 ... 0.   0.   0.  ]
 [0.05 0.05 0.05 ... 0.   0.   0.  ]]
